In [49]:
import pandas as pd
import numpy as np
import sklearn

In [50]:
df_no = pd.read_excel("data_washed/norway_smokers.xlsx").drop(columns=["Unnamed: 0"])
df_no = df_no.rename(columns={"Unnamed: 2": "Year", "Unnamed: 3": "smokers_percentage"})
df_no["Year"] = pd.to_numeric(df_no["Year"], errors="coerce")
df_no = df_no.dropna(subset=["Year", "smokers_percentage"]).reset_index(drop=True)
df_no



In [73]:
df_no1 = pd.read_excel("norge_royk1.xlsx")
df_no2 = pd.read_excel("norge_royk2.xlsx")


In [74]:
# ...existing code...
# drop first two columns, drop first two rows, reset index
df_no1 = df_no1.drop(columns=["Unnamed: 0", "Unnamed: 1"]).iloc[2:].reset_index(drop=True)

# correct rename using the `columns=` dict
df_no1 = df_no1.rename(columns={"Unnamed: 2": "Year", "Unnamed: 3": "Smokers"})

# ensure Year numeric and drop rows with missing Year or Smokers
df_no1["Year"] = pd.to_numeric(df_no1["Year"], errors="coerce")
df_no1 = df_no1.dropna(subset=["Year", "Smokers"]).reset_index(drop=True)

df_no1
# ...existing code...

In [75]:
# ...existing code...
# drop first two columns, drop first two rows, reset index
df_no2 = df_no2.drop(columns=["Unnamed: 0", "Unnamed: 1"]).iloc[2:].reset_index(drop=True)

# correct rename using the `columns=` dict
df_no2 = df_no2.rename(columns={"Unnamed: 2": "Year", "Unnamed: 3": "Smokers"})

# ensure Year numeric and drop rows with missing Year or Smokers
df_no2["Year"] = pd.to_numeric(df_no2["Year"], errors="coerce")
df_no2 = df_no2.dropna(subset=["Year", "Smokers"]).reset_index(drop=True)

df_no2
# ...existing code...

In [79]:
df_no = df_no1.merge(df_no2, on="Year")
df_no["Smokers"] = df_no["Smokers_x"] + df_no["Smokers_y"]
df_no = df_no.drop(columns=["Smokers_y", "Smokers_x"])

In [80]:
df_no

In [83]:
df_no = pd.read_excel("data_washed/norway_smokers.xlsx").drop(columns=["Unnamed: 0"])
df_no = df_no.rename(columns={"Unnamed: 2": "Year", "Unnamed: 3": "smokers_percentage"})
df_no["Year"] = pd.to_numeric(df_no["Year"], errors="coerce")
df_no = df_no.dropna(subset=["Year", "smokers_percentage"]).reset_index(drop=True)
df_no


In [ ]:
df_sp = pd.read_excel("roykere_danmark.xlsx")
df_sp = df_sp.rename(columns={"Percentage": "smokers_percentage"}).drop(columns=["Unnamed: 0"])


df_sp

In [85]:
df_merged = df_sp.merge(df_no, how="left", on="Year", suffixes=("_sp", "_no"))

# Example: rename for clarity or combine columns if you want a single smokers_percentage column
# df_merged["smokers_percentage_no"] = df_merged["smokers_percentage_no"].astype(float)
# df_merged["smokers_percentage_sp"] = df_merged["smokers_percentage_sp"].astype(float)
df = df_merged.head(12)
df

In [86]:

# ...existing code...
import statsmodels.api as sm

# Prepare long panel for DiD using the merged data (df_merged)
# create separate country-year rows from the merged wide frame
df_sp_long = df[['Year', 'smokers_percentage_sp']].rename(
    columns={'smokers_percentage_sp': 'smokers_percentage'}
).assign(Country='spmark')

df_no_long = df[['Year', 'smokers_percentage_no']].rename(
    columns={'smokers_percentage_no': 'smokers_percentage'}
).assign(Country='Norway')

df_long = pd.concat([df_sp_long, df_no_long], ignore_index=True)

# clean and types
df_long['Year'] = pd.to_numeric(df_long['Year'], errors='coerce')
df_long['smokers_percentage'] = pd.to_numeric(df_long['smokers_percentage'], errors='coerce')
df_long = df_long.dropna(subset=['Year', 'smokers_percentage']).reset_index(drop=True)

# DiD variables: Norway indicator, Post2004, and interaction (Treated)
df_long['Norway'] = (df_long['Country'] == 'Norway').astype(int)
df_long['Post2004'] = (df_long['Year'] >= 2004).astype(int)
df_long['Treated'] = df_long['Norway'] * df_long['Post2004']

# Run OLS DiD: smokers_percentage ~ Norway + Post2004 + Treated
X = df_long[['Norway', 'Post2004', 'Treated']]
X = sm.add_constant(X)
y = df_long['smokers_percentage']

model = sm.OLS(y, X).fit()
print("\n" + "="*60)
print("DiD: Norway workplace smoking ban (2004) — outcome: smokers_percentage")
print("="*60)
print(model.summary())

# key result
did_coef = model.params['Treated']
did_p = model.pvalues['Treated']
print(f"\nDiD estimate (Treated): {did_coef:.4f}, p-value: {did_p:.4f}")
# ...existing code...